# Info
Name:  

    Prepare_ORACLES_lut_reflectance

Purpose:  

    Create the input libradtran files for creating a lut of low clouds with aerosol on top to be used in ORACLES operational 
    cloud retrievals. Using the reflectance values calculated from SSFR measurements. Creating the lut for Irradiance values.

Calling Sequence:

    python Prepare_ORACLES_lut_reflectance
  
Input:

    none

Output:
   
    input files for libradtran 2.0 (uvspec) 
  
Keywords:

    none
  
Dependencies:

    - numpy
    - scipy : for saving and reading
    - mplt_toolkits for basemap, map plotting
    - pdb
    - datetime

  
Needed Files:

  - aero_file_v4.txt details of the aerosol layer above
    
History:

    Written: Samuel LeBlanc, Santa Cruz, CA, 2018-06-25
             based on Prepare_ORACLES_lut
    Modified: Samuel LeBlanc, Santa Cruz, CA, 2020-11-18
             updates for new Mie scattering calculations: v7_irr
    

# Prepare the python environment

In [1]:
import numpy as np
import scipy.io as sio
import os
import Run_libradtran as RL

In [2]:
from load_utils import load_from_json
from path_utils import getpath
from tqdm.notebook import tqdm 

In [3]:
fp = getpath('ORACLES')
fp_rtm = getpath('rtm')
fp_uvspec = getpath('uvspec')+'uvspec'
fp_rtmdat = fp_rtm+'dat/'
fp_uvspec_dat = getpath('uvspec_dat')

Return path named:  ORACLES /data/sam/ORACLES/
Return path named:  rtm /scratch/rtm/
Return path named:  uvspec /home/sam/libradtran/libRadtran-2.0.2/bin/
Return path named:  uvspec_dat /home/sam/libradtran/libRadtran-2.0.2/data/


In [6]:
if os.sys.platform == 'win32':
    fp = 'C:\\Users\\sleblan2\\Research\\ORACLES\\'
    fp_rtm = 'C:\\Users\\sleblan2\\Research\\ORACLES\\rtm\\'
    fp_uvspec = 'C:\\Users\\sleblan2\\Research\\libradtran\\libRadtran-2.0-beta\\bin\\uvspec'
    fp_rtmdat = 'C:\\Users\\sleblan2\\Research\\libradtran\\libRadtran-2.0-beta\\data\\'
elif os.sys.platform == 'linux2':
    fp = '/u/sleblan2/ORACLES/'
    fp_rtm = '/nobackup/sleblan2/rtm/'
    fp_uvspec = '/u/sleblan2/libradtran/libRadtran-2.0-beta/bin/uvspec'
    fp_rtmdat = '/nobackup/sleblan2/AAC_DARF/rtm/' #'/u/sleblan2/4STAR/rtm_dat/'
else:
    raise Exception

# Setup the variables used to create the lut

In [4]:
vv = 'v7_irr'
mu = np.arange(1.02,3.4,0.05)
mu.shape

(48,)

In [5]:
sza = np.arccos(1.0/mu)*180.0/np.pi
#sza = np.arange(40,91,5)
print(sza)

[11.36487697 20.83969366 26.76550058 31.2732978  34.94801372 38.05665663
 40.74905438 43.11972483 45.23300465 47.13508975 48.86048959 50.43578157
 51.88193569 53.21582641 54.451256   55.5996703  56.67067347 57.67240684
 58.61183354 59.49495605 60.3269852  61.11247301 61.85541869 62.55935386
 63.22741194 63.86238505 64.46677106 65.04281282 65.59253108 66.11775223
 66.62013188 67.10117502 67.56225324 68.00461964 68.4294217  68.83771245
 69.23046027 69.60855741 69.97282752 70.32403224 70.6628771  70.99001664
 71.30605908 71.61157038 71.90707798 72.193074   72.47001827 72.73834093]


In [8]:
tau = np.array([0.1,0.2,0.5,0.75,1.0,1.5,2.0,3.0,4.0,5.0,
       6.0,7.0,8.0,9.0,10.0,12.5,15.0,17.5,20.0,25.0,30.0,35.0,40.0,50.0,
       60.0,80.0,100.0])
#ref = np.append(np.append(np.arange(1,15),np.arange(15,30,2)),np.ceil(np.arange(30,61,2.5)))
ref = np.arange(0.5,20.5,0.5)

In [9]:
ref

array([ 0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,
        6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. ,
       11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. , 16.5,
       17. , 17.5, 18. , 18.5, 19. , 19.5, 20. ])

In [10]:
print(ref.shape)
print(tau.shape)

(40,)
(27,)


In [169]:
pmom = RL.make_pmom_inputs(fp_rtm=fp_rtmdat,source='solar',deltascale=True, new=True)

In [170]:
pmom['file_name']

[u'/scratch/rtm/dat/mie_run2020112_SL.mat']

In [171]:
aero = load_from_json(fp+'aero_file_v4.txt')

In [172]:
aero['ext']

array([[0.333445  , 0.26344564, 0.23560441, 0.17216642, 0.11197813,
        0.06053209, 0.049     , 0.04588309, 0.03490222, 0.021     ,
        0.014     , 0.007     ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [173]:
if 'ext' in aero: print 'a'

a


In [174]:
#geo = {'lat':-22.979,
#       'lon':14.645,
#       'doy':245,
#       'zout':[0.2,1.5,100.0]}
geo = {'lat':-16.0,
       'lon': 9.0,
       'doy':253, #September 10th
       'zout':[0.2,1.5,100.0]}
#aero = {'z_arr':[2.0,5.0],
#        'ext':np.array([[0.6,0.4,0.10,0.04],[0.0,0.0,0.0,0.0]]),
#        'ssa':np.array([[0.8,0.85,0.9,0.95],[0.9,0.9,0.9,0.9]]),
#        'asy':np.array([[0.8,0.8,0.8,0.8],[0.8,0.8,0.8,0.8]]),
#        'wvl_arr':[400.0,500.0,650.0,940.0],
#        'disort_phase':False,
#        'expand_hg':True}
cloud = {'ztop':1.0,
         'zbot':0.5,
         'write_moments_file':True,
         'moms_dict':pmom}
source = {'wvl_range':[350,1650],
          'source':'solar',
          'integrate_values':False,
          'run_fuliou':False,
          'dat_path':fp_uvspec_dat,
          'atm_file':fp_uvspec_dat + 'atmmod/afglt.dat',
          'zenith':False}
albedo = {'create_albedo_file':False,
          'sea_surface_albedo':True,
          'wind_speed':5.0}

In [175]:
pmom['file_name']

[u'/scratch/rtm/dat/mie_run2020112_SL.mat']

In [176]:
RL.print_version_details(fp+'ORACLES_lut_%s.txt'%vv,vv,geo=geo,
                         aero=aero,cloud=cloud,source=source,albedo=albedo,
                         tau=tau,ref=ref,sza=sza,cloud_pmom_file=pmom['file_name'])

In [178]:
fp_in = os.path.join(fp_rtm,'input','%s_ORACLES'%vv)
fp_out = os.path.join(fp_rtm,'output','%s_ORACLES'%vv)

In [179]:
f_slit_vis = os.path.join(fp_rtm,'vis_1nm.dat')
f_slit_nir = os.path.join(fp_rtm,'nir_1nm.dat')

In [180]:
if not os.path.exists(fp_in):
    os.makedirs(fp_in)
if not os.path.exists(fp_out):
    os.makedirs(fp_out)

In [21]:
f_list = open(os.path.join(fp_rtm,'ORACLES_list_%s.sh'%vv),'w')
print f_list.name

/scratch/rtm/ORACLES_list_v7_irr.sh


In [22]:
def isjupyter():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

## Run through typical method

In [32]:
if isjupyter():
    pbar = tqdm(total=len(sza)*len(tau)*len(ref))
for s in sza:
    for t in tau:
        for r in ref:
            fname = 'lut_irr_sza%04.1f_tau%06.2f_ref%04.1f' % (s,t,r)
            geo['sza'] = s
            cloud['tau'] = t
            cloud['ref'] = r
            if False: #r>=5.0:
                cloud['phase'] = 'ic'
                fname0 = fname+'_'+cloud['phase']+'_w0.dat'
                source['wvl_range'] = [499.0,501.]
                source['slit_file'] = f_slit_vis
                RL.write_input_aac(os.path.join(fp_in,fname0),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
                f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname0)+' > '+os.path.join(fp_out,fname0)+'\n')
                fname1 = fname+'_'+cloud['phase']+'_w1.dat'
                source['wvl_range'] = [981.,1700.]
                source['slit_file'] = f_slit_nir
                RL.write_input_aac(os.path.join(fp_in,fname1),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
                f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname1)+' > '+os.path.join(fp_out,fname1)+'\n')
            if r<=20.0:
                cloud['phase'] = 'wc'
                fname0 = fname+'_'+cloud['phase']+'_w0.dat'
                source['wvl_range'] = [500.0,500.0]
                source['slit_file'] = f_slit_vis
                RL.write_input_aac(os.path.join(fp_in,fname0),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
                f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname0)+' > '+os.path.join(fp_out,fname0)+'\n')
                fname1 = fname+'_'+cloud['phase']+'_w1.dat'
                source['wvl_range'] = [1650.0,1650.0]
                source['slit_file'] = f_slit_nir
                RL.write_input_aac(os.path.join(fp_in,fname1),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
                f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname1)+' > '+os.path.join(fp_out,fname1)+'\n')                
            if isjupyter(): 
                pbar.update(1)
            else:
                print s,t,r
        #break
    #break

A Jupyter Widget

KeyboardInterrupt: 

In [25]:
f_list.close()

## Run for multiprocessing

In [181]:
from multiprocessing import Pool, cpu_count
from copy import deepcopy
import signal
import warnings
warnings.simplefilter('ignore')

In [182]:
def worker_init(verbose=True):
    # ignore the SIGINI in sub process, just print a log
    def sig_int(signal_num, frame):
        if verbose: 
            print 'signal: %s' % signal_num
        raise IOError
    signal.signal(signal.SIGINT, sig_int)

In [184]:
f_list = open(fp_rtm+'{}_{}.sh'.format(name,vv),'w')
print f_list.name
fpp_in = fp_rtm+'input/{}_{}/'.format(name,vv)
fpp_out = fp_rtm+'output/{}_{}/'.format(name,vv)

/scratch/rtm/ORACLES_v7_irr.sh


In [185]:
if not os.path.exists(fp_in):
    os.makedirs(fp_in)
if not os.path.exists(fp_out):
    os.makedirs(fp_out)

In [186]:
if isjupyter():
    pbar = tqdm(total=len(sza)*len(tau)*len(ref))
bb = []
for s in sza:
    for t in tau:
        for r in ref:
            fname = 'lut_irr_sza%04.1f_tau%06.2f_ref%04.1f' % (s,t,r)
            geo['sza'] = s
            cloud['tau'] = t
            cloud['ref'] = r
            
            cloud['phase'] = 'wc'
            fname0 = fname+'_'+cloud['phase']+'_w0.dat'
            source['wvl_range'] = [500.0,500.0]
            source['slit_file'] = f_slit_vis
            #RL.write_input_aac(os.path.join(fp_in,fname0),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
            #                   verbose=False,make_base=False,set_quiet=True)
            f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname0)+' > '+os.path.join(fp_out,fname0)+'\n')
            fname1 = fname+'_'+cloud['phase']+'_w1.dat'
            source['wvl_range'] = [1650.0,1650.0]
            source['slit_file'] = f_slit_nir
            #RL.write_input_aac(os.path.join(fp_in,fname1),geo=geo,aero=aero,cloud=cloud,source=source,albedo=albedo,
            #                   verbose=False,make_base=False,set_quiet=True)
            f_list.write(fp_uvspec+' < '+os.path.join(fp_in,fname1)+' > '+os.path.join(fp_out,fname1)+'\n')
            
            bb.append({'geo':deepcopy(geo),'cod':t,'ref':r,
                       'f0':deepcopy(fname0),'f1':deepcopy(fname1),'w0':[500.0,500.0],'w1':[1650.0,1650.0],
                       's0':f_slit_vis,'s1':f_slit_nir})
            
            if isjupyter(): 
                pbar.update(1)
            else:
                print s,t,r
f_list.close()

A Jupyter Widget

In [187]:
def write_files(d,cloud=cloud,source=source,albedo=albedo,aero=aero,fp_in=fp_in):
    'function to feed the pool of workers to write out the all the files'
    cloud['tau'],cloud['ref'] = d['cod'],d['ref']
    source['wvl_range'] = d['w0']
    source['slit_file'] = d['s0']
    RL.write_input_aac(fp_in+'/'+d['f0'],geo=d['geo'],aero={},cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)
    source['wvl_range'] = d['w1']
    source['slit_file'] = d['s1']
    RL.write_input_aac(fp_in+'/'+d['f1'],geo=d['geo'],aero={},cloud=cloud,source=source,albedo=albedo,
                                   verbose=False,make_base=False,set_quiet=True)

In [188]:
p = Pool(cpu_count()-1,worker_init)

In [189]:
len(bb)

51840

In [190]:
results = []
max_ = len(bb)
with tqdm(total=max_) as pbar:
    for i, res in tqdm(enumerate(p.imap_unordered(write_files, bb))):
        pbar.update()
        results.append(res)

A Jupyter Widget

A Jupyter Widget

In [191]:
results[0]

# Run the calculations

In [193]:
f_list = fp_rtm+'{}_{}.sh'.format(name,vv)

In [194]:
! wc -l $f_list

103680 /scratch/rtm/ORACLES_v7_irr.sh


In [195]:
f_listout = f_list+'.out'

In [196]:
!parallel --jobs=22 --bar < $f_list #2> $f_listout

18.33544 sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in< /scrat012345679inininininin2.0.2b/bin/uvspec < /scratch74691358.0.2b/bin/uvspec < /scratchininn.0.2b/bin/uvspec < /scratchin2.0.2b/bin/uvspec < /scratchininininininin2.0.2b/bin/uvspec < /scratch.0.2b/bin/uvspec < /scratch/rtm/inin649691358inin-2.0.2b/bin/uvspec < /scratinininininin.2b/bin/uvspec < /scrat-2.0.2b/bin/uvspec < /scratibradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inininin2.2627314814815am/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat-2.0.2b/bin/uvspec < /scratome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inn-2.0.2b/bin/uvspec < /scratn0.2b/bin/uvspec < /scratch/rtm/inin0.2b/bin/uvspec < /scratch/rtm/ininn-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scratme/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat13% 13515:90165=1214s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrats /home/sam/libradtran/libRadtran-2.0.2b/bin/uvs

21%833333333me/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in< /scratn-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrat83487654321< /scratch/rtm/inn/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/ininin.0.2b/bin/uvspec < /scratch/rtm/inome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inn-2.0.2b/bin/uvspec < /scratam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrat0.2b/bin/uvspec < /scratch/rtm/in/home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratnininn-2.0.2b/bin/uvspec < /scrats /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrat /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrats /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat% 19273:84407=1143s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrats /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec <

24% 25096:78584=1067s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat-2.0.2b/bin/uvspec < /scrat1.3281250.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scratinec < /scrat< /scratch/rtm/inin6149691358uvspec < /scratn-2.0.2b/bin/uvspec < /scratin/uvspec < /scratch/rtm/in/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inn-2.0.2b/bin/uvspec < /scrat0.2b/bin/uvspec < /scratch/rtm/in /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratin0.2b/bin/uvspec < /scratch/rtm/in /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratin/uvspec < /scratch/rtm/inm/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratc < /scrattran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scratch/rtm/in-2.0.2b/bin/uvspec < /scratibradtran/libRadtran-2.0.2b/bin/uvspec < /scratRadtran-2.0.2b/bin/uvspec < /scratch/rtm/ininn-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scrat0.2b/bin/uvspec < /scratch/rtm/in637345679pec < /scratch/rtm/inn0.2b/bin/uvspec < /scratch/rtm/in /home/sam/libradtran/

26% 27471:76209=1037s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratn-2.0.2b/bin/uvspec < /scratin/uvspec < /scratch/rtm/inradtran/libRadtran-2.0.2b/bin/uvspec < /scratinn-2.0.2b/bin/uvspec < /scratome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/intran/libRadtran-2.0.2b/bin/uvspec < /scrat78525=1067s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat#   1067 sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in/bin/uvspec < /scratn/uvspec < /scratch/rtm/insam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratnbin/uvspec < /scrath/rtm/in /scratch/rtm/inn/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in7s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratc /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/ininn-2.0.2b/bin/uvspec < /scrat0.2b/bin/uvspec < /scratch/rtm/in< /scratch/rtm/inininn-2.0.2b/bin/uvspec < /scrat /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat/libradtran/libRadtran-2.0.2b/bin/uvspec <

#   1013 sec /home/sam/libradtranme/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat2b/bin/uvspec < /scratch/rtm/inome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratin2.0.2b/bin/uvspec < /scratm0m /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat7901spec < /scratch/rtm/in sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inin.0.2b/bin/uvspec < /scrat2b/bin/uvspec < /scratch/rtm/in/scratch/rtm/inin.2b/bin/uvspec < /scrat/scratn-2.0.2b/bin/uvspec < /scrat0.2b/bin/uvspec < /scratch/rtm/in /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scrat< /scratch/rtm/inan/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/insec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in037037uvspec < /scratinn-2.0.2b/bin/uvspec < /scratec < /scratdtran-2.0.2b/bin/uvspec < /scratch/rtm/in sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/in240740741in/uvspec < /scratn-2.0.2b/bin/uvspec < /scrat.0.2b/bin/uvspec < /scratch/rtm/inm /home/sam/

...

#   96 sec /home031=96s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/bin/uvspec < /scratch2b/bin/uvspec < /scratch/rtm/inpsam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratche/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inp6790123457pec < /scratchvspec < /scratch/rtm/inpadtran/libRadtran-2.0.2b/bin/uvspec < /scratchbradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inp% 99652:4028=127s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchchm/inp.0.2b/bin/uvspec < /scratch2.0.2b/bin/uvspec < /scratch/rtm/inpme/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchhome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inp3333atch/bin/uvspec < /scratch2b/bin/uvspec < /scratch/rtm/inpsam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inp 99720:3960=125s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchhrtm/inp-2.0.2b/bin/uvspec < /scratchan-2.0.2b/bin/uvspec < /scrat

#   66 sec /home072=66s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchratchratch/rtm/inpuRadtran-2.0.2b/bin/uvspec < /scratchbRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu/home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratcham/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu7m97% 100694:2986=95s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchtchtch/rtm/inpudtran-2.0.2b/bin/uvspec < /scratchadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu3=94s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu18132716049vspec < /scratch/scratch/rtm/inpu2.0.2b/bin/uvspec < /scratch.0.2b/bin/uvspec < /scratch/rtm/inpuome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchome/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu7161 /scratch/rtm/inpun-2.0.2b/bin/uvspec < /scratchan-2.0.2b/bin/uvspec < /scratch/rtm/inpus /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec <

98.946759259259e/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpuatchcratchscratch/rtm/inpuibRadtran-2.0.2b/bin/uvspec < /scratchibRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu040=65s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch  65 sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu8.0372299382716in/uvspec < /scratchbin/uvspec < /scratch/rtm/inpum/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchadtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu:2023=65s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch   64 sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpubin/uvspec < /scratchvspec < /scratch/rtm/inputran/libRadtran-2.0.2b/bin/uvspec < /scratchdtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu101684:1996=64s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchpub/bin/uvspec < /scratch2b/bin/uvspec < /scratch/rtm/inpu/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchadtran/li

#   2 sec /home/sam/ /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtcratch/rtm/inputran-2.0.2b/bin/uvspec < /scratchdtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu34s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch sec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchn/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpu28:1052=34s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchub/bin/uvspec < /scratch/bin/uvspec < /scratch/rtm/inpuam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratchsam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inpuatchinpu2b/bin/uvspec < /scratch.2b/bin/uvspec < /scratch/rtm/inpuubin/uvspec < /scratch/b/bin/uvspec < /scratch/rtm/inpuam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/2=31s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/pec < /scratch/rtm/inpubRadtran-2.0.2b/bin/uvspec < /scratch/libRadtran-2.0.2b/bin/uvspec <

100% 103680:0=0s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtsec /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/input740741/scratch/rt /scratch/rtm/inputibRadtran-2.0.2b/bin/uvspec < /scratch/rtn/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/input640:40=1s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtmput/bin/uvspec < /scratch/rt.2b/bin/uvspec < /scratch/rtm/inputtin/uvspec < /scratch/rtb/bin/uvspec < /scratch/rtm/input/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtm/inputm99% 103675:5=0s /home/sam/libradtran/libRadtran-2.0.2b/bin/uvspec < /scratch/rtmmh/rtm/input
